In [171]:
import tensorflow as tf
import pickle
from tensorflow.keras import Sequential
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [172]:
with open('countvectorizer.pkl', 'rb') as f:
    count_vectorizer = pickle.load(f)

In [173]:
sentiment_mapping={
    '0':'sadness',
    '1':'worry',
    '2':['anger','hate'],
    '3':['neutral','boredom'],
    '4':['enthusiasm','surprise','fun'],
    '5':'relief',
    '6':'love',
    '7':'happiness'
}

In [174]:
X=['makes feel wonderful']

In [175]:
X=count_vectorizer.transform(X).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0]])

In [176]:
_,m=X.shape

In [177]:
opt=tf.keras.optimizers.Adam()
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
batch_size=1000

In [178]:
model=Sequential(
      [tf.keras.Input(shape=(m,)),
      tf.keras.layers.Dense(units=3000,activation='relu'),
      tf.keras.layers.Dense(units=600,activation='relu'),
      tf.keras.layers.Dense(units=400,activation='relu'),
      tf.keras.layers.Dense(units=260,activation='relu'),
      tf.keras.layers.Dense(units=30,activation='relu'),
      tf.keras.layers.Dense(units=8,activation='linear')])
    
model.compile(loss=loss,optimizer=opt,metrics='accuracy')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 3000)              51525000  
                                                                 
 dense_19 (Dense)            (None, 600)               1800600   
                                                                 
 dense_20 (Dense)            (None, 400)               240400    
                                                                 
 dense_21 (Dense)            (None, 260)               104260    
                                                                 
 dense_22 (Dense)            (None, 30)                7830      
                                                                 
 dense_23 (Dense)            (None, 8)                 248       
                                                                 
Total params: 53678338 (204.77 MB)
Trainable params: 5

In [179]:
model.load_weights('model.h5')

In [180]:
def clean_data(text):
    #removing @ tags 
    text=re.sub(r'@[a-zA-z0-9]+\s*',' ',str(text))
    
    #removing urls
    text=re.sub(r'http\S+', '', text)
    text=re.sub(r'www\.\S+', '', text)
    
    #replacing multiple whitesapces by a single
    text = re.sub(r'\s+',' ',text)
    
    # remove all single characters(surrounded by whitespace)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # remove all single characters except i and I (surrounded by whitespace)
    # text = re.sub(r'\s+(?![iI])[a-zA-Z]\s+', ' ', text)
    
    
    # Converting to Lowercase 
    text = text.lower()
    
    # Lemmatization- splits into list of words ['The', 'quick', ....]
    text = text.split()

    lemma = WordNetLemmatizer()
    text = [lemma.lemmatize(word) for word in text]
    text = ' '.join(text)
    
    words = nltk.word_tokenize(text)
    
    # Get the list of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords from the text
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Recreate the text without stopwords
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

In [202]:
X='i want to dance'
X=[clean_data(X)]
print(X)
X=count_vectorizer.transform(X).toarray()
X

['want dance']


array([[0, 0, 0, ..., 0, 0, 0]])

In [203]:
y=model.predict(X)
sentiment_mapping[str(np.argmax(y))]

1/1 [==============================] - 0s 23ms/step


['neutral', 'boredom']

In [204]:
y_sm=tf.nn.softmax(y)
y_sm

<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[2.51490701e-06, 1.22948736e-03, 3.55287193e-05, 9.98312950e-01,
        1.23260485e-04, 1.28640965e-08, 8.51897930e-09, 2.96186074e-04]],
      dtype=float32)>

In [184]:
pred=np.argmax(y_sm)
pred

1